# Efficiently train Large Language Models with LoRA and Hugging Face

In this blog, we are going to show you how to apply [Low-Rank Adaptation of Large Language Models (LoRA)](https://arxiv.org/abs/2106.09685) to fine-tune FLAN-T5 XXL (11 billion parameters) on a single GPU. We are going to leverage Hugging Face [Transformers](https://huggingface.co/docs/transformers/index), [Accelerate](https://huggingface.co/docs/accelerate/index), and [PEFT](https://github.com/huggingface/peft). 

You will learn how to:

1. Setup Development Environment
2. Load and prepare the dataset
3. Fine-Tune T5 with LoRA and bnb int-8
4. Evaluate & run Inference with LoRA FLAN-T5
5. Cost performance comparison

### Quick intro: PEFT or Parameter Efficient Fine-tunin

[PEFT](https://github.com/huggingface/peft), or Parameter Efficient Fine-tuning, is a new open-source library from Hugging Face to enable efficient adaptation of pre-trained language models (PLMs) to various downstream applications without fine-tuning all the model's parameters. PEFT currently includes techniques for:

- LoRA: [LORA: LOW-RANK ADAPTATION OF LARGE LANGUAGE MODELS](https://arxiv.org/pdf/2106.09685.pdf)
- Prefix Tuning: [P-Tuning v2: Prompt Tuning Can Be Comparable to Fine-tuning Universally Across Scales and Tasks](https://arxiv.org/pdf/2110.07602.pdf)
- P-Tuning: [GPT Understands, Too](https://arxiv.org/pdf/2103.10385.pdf)
- Prompt Tuning: [The Power of Scale for Parameter-Efficient Prompt Tuning](https://arxiv.org/pdf/2104.08691.pdf)

*Note: This tutorial was created and run on a g5.2xlarge AWS EC2 Instance, including 1 NVIDIA A10G.*

## 1. Setup Development Environment

In our example, we use the [PyTorch Deep Learning AMI](https://docs.aws.amazon.com/dlami/latest/devguide/tutorial-pytorch.html) with already set up CUDA drivers and PyTorch installed. We still have to install the Hugging Face Libraries, including transformers and datasets. Running the following cell will install all the required packages.

In [1]:
import numpy as np
from scipy.special import softmax
import pdb
import pandas as pd
import math
from typing import List
import random
import argparse
import torch


def sent_scoring(model_tokenizer, text, cuda, score_type="loss", output_attentions=False, length_normalize=False):
    model = model_tokenizer[0]
    tokenizer = model_tokenizer[1]
    assert model is not None
    assert tokenizer is not None
    encoded_text = tokenizer.encode(text)
    input_ids = torch.tensor(encoded_text).unsqueeze(0)
    if cuda:
        input_ids = input_ids.to('cuda')
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids, output_attentions=output_attentions)
    loss, logits = outputs[:2]

    sentence_prob = loss.item()
    if score_type == "prob":
        if length_normalize:
            mult = 2
        else:
            mult = len(encoded_text)

        sentence_prob = math.exp(-1.0 * loss * (mult - 1))

    if output_attentions:
        attn = outputs["attentions"]
        return sentence_prob, attn, input_ids

    return sentence_prob

def confusion_matrix(P_forward_1, P_forward_2, P_backward_1, P_backward_2):
    correct_forward = len(np.where(np.array(P_forward_1) >= 0.5)[0]) + len(np.where(np.array(P_forward_2) >=0.5)[0])
    wrong_forward = len(P_forward_1) + len(P_forward_2) - correct_forward

    correct_backward = len(np.where(np.array(P_backward_1) >= 0.5)[0]) + len(np.where(np.array(P_backward_2) >=0.5)[0])
    wrong_backward = len(P_backward_1) + len(P_backward_2) - correct_backward

    print("correct forward", correct_forward, "wrong forward", wrong_forward, "correct backward", correct_backward, "wrong_backward", wrong_backward)

    results = {
        "correct_forward": correct_forward,
        "wrong_forward": wrong_forward,
        "correct_backward": correct_backward,
        "wrong_backward": wrong_backward
    }

    return results

from tqdm import tqdm

def evaluate_model(model, tokenizer, test_set, middle_phrase="", use_prefix=0, verbose=True, score_type="prob", use_cuda=False, return_acc=False, total = 1094) -> tuple:
    preds = []
    labels = []
    x_1 = []
    x_2 = []
    y_1 = []
    y_2 = []
    P_x_1 = []
    P_x_2 = []
    P_y_1 = []
    P_y_2 = []
    P_x_1_y_1 = []
    P_x_1_y_2 = []
    P_x_2_y_1 = []
    P_x_2_y_2 = []
    P_x_1_correct = []
    P_x_2_correct = []
    P_y_1_correct = []
    P_y_2_correct = []
    correct = 0

    for i, metaphor_data in tqdm(enumerate(test_set), total = total):
        ctx, p1, p2 = metaphor_data["startphrase"], metaphor_data["ending1"], metaphor_data["ending2"]
        labels.append(int(metaphor_data["labels"]))
        if use_prefix > 0:
            prefix_prompt = select_prefix_prompts(prompt_file, use_prefix) if use_prefix else ""
        else:
            prefix_prompt = ""

        sent1 = prefix_prompt + ctx + ". " + middle_phrase + p1 + "."
        sent2 = prefix_prompt + ctx + ". " + middle_phrase + p2 + "."

        score1 = sent_scoring((model, tokenizer), sent1, use_cuda, score_type=score_type)
        score2 = sent_scoring((model, tokenizer), sent2, use_cuda, score_type=score_type)

        if score_type == "loss":
            pred = 0 if score1 < score2 else 1
        else:
            pred = 1 if score1 < score2 else 0

        pred_sent = sent1 if pred == 0 else sent2

        if i % 2 == 0:
            x_1.append(ctx)
            x_1_score = sent_scoring((model, tokenizer), ctx + ".", use_cuda, score_type=score_type)
            P_x_1.append(x_1_score)
            y_1.append(p1)
            y_2.append(p2)
            y1_score = sent_scoring((model, tokenizer), p1 + ".", use_cuda, score_type=score_type)
            y2_score = sent_scoring((model, tokenizer), p2 + ".", use_cuda, score_type=score_type)
            P_y_1.append(y1_score)
            P_y_2.append(y2_score)

            P_x_1_y_1.append(score1)
            P_x_1_y_2.append(score2)
            P_x_1_correct.append(score1/(score1 + score2))

        else:
            x_2.append(ctx)
            x_2_score = sent_scoring((model, tokenizer), ctx + ".", use_cuda, score_type=score_type)
            P_x_2.append(x_2_score)
            P_x_2_y_1.append(score1)
            P_x_2_y_2.append(score2)
            P_x_2_correct.append(score2/(score1 + score2))

            P_y_1_correct.append(P_x_1_y_1[-1]/(P_x_1_y_1[-1] + score1))
            P_y_2_correct.append(score2/(P_x_1_y_2[-1] + score2))

        if verbose:
            print(f"Q: {ctx}: 1. {p1} 2. {p2}")
            print(f"model says '{pred_sent}' is more likely")
            print("\n")
        if pred == metaphor_data["labels"]:
            correct += 1
        preds.append(pred)

    cols = {"x_1": x_1, "x_2": x_2, "y_1": y_1, "y_2": y_2, "P(x_1)": P_x_1, "P(x_2)": P_x_2, "P(y_1)": P_y_1, "P(y_2)": P_y_2,
        "P(x_1, y_1)": P_x_1_y_1, "P(x_1, y_2)": P_x_1_y_2, "P(x_2, y_1)": P_x_2_y_1, "P(x_2, y_2)": P_x_2_y_2,
        "P(y_1|x_1)": P_x_1_correct, "P(y_2|x_2)": P_x_2_correct, "P(x_1|y_1)": P_y_1_correct, "P(x_2|y_2)": P_y_2_correct}
    out_df = pd.DataFrame(cols)

    if return_acc:
        return correct/len(preds), out_df, preds, labels

    return out_df, preds, labels

def compute_stats(total_df: pd.DataFrame, all_preds: List, all_labels: List) -> None:
    print("overall accuracy: ")
    accuracyy = len(np.where(np.array(all_preds) == np.array(all_labels))[0])/len(all_labels)
    print(accuracyy)
    print("confusion matrix: ")
    matrix_dic = confusion_matrix(list(total_df["P(y_1|x_1)"]), list(total_df["P(y_2|x_2)"]), list(total_df["P(x_1|y_1)"]), list(total_df["P(x_2|y_2)"]))

    return accuracyy, matrix_dic


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
!pip uninstall datasets -y
!pip install datasets

Found existing installation: datasets 2.1.0
Uninstalling datasets-2.1.0:
  Successfully uninstalled datasets-2.1.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 9.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.9.0
    Uninstalling fsspec-2023.9.0:
      Successfully uninstalled fsspec-2023.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.0.2 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is 

In [3]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

In [4]:
# install Hugging Face Libraries
#!pip install "peft==0.2.0"
#!pip install "transformers==4.27.1" "datasets==2.9.0" "accelerate==0.17.1" "evaluate==0.4.0" "bitsandbytes==0.37.1" loralib --upgrade --quiet
# install additional dependencies needed for training
!pip install rouge-score tensorboard py7zr 

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 66.6 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 7.5 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=8ad5a20bd19b6397f4f44ab0647ae2d97c7e56f7676f72e062b65b3ffb86d4f0
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


## 2. Load and prepare the dataset

we will use the [samsum](https://huggingface.co/datasets/samsum) dataset, a collection of about 16k messenger-like conversations with summaries. Conversations were created and written down by linguists fluent in English.

```python
{
  "id": "13818513",
  "summary": "Amanda baked cookies and will bring Jerry some tomorrow.",
  "dialogue": "Amanda: I baked cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)"
}
```

To load the `samsum` dataset, we use the **`load_dataset()`** method from the 🤗 Datasets library.

In [5]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Load dataset from the hub
dataset = load_dataset("nightingal3/fig-qa")

print(f"Validation dataset size: {len(dataset['validation'])}")

# %%
model_id="google/flan-t5-large"

# Load tokenizer of FLAN-t5-XL
tokenizer = AutoTokenizer.from_pretrained(model_id)

# %%
def preprocess_function(sample, padding="max_length"):
    # Your startphrase will be the input and the correct ending will be your target
    inputs = sample['startphrase']
    
    # Choose the correct ending based on the labels value for each sample in the batch
    targets = [sample['ending1'][i] if sample['labels'][i] == 0 else sample['ending2'][i] for i in range(len(sample['labels']))]

    # Tokenize inputs
    model_inputs = tokenizer(inputs, max_length=50, padding=padding, truncation=True)
    
    # Tokenize targets
    labels = tokenizer(targets, max_length=50, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Applying the preprocessing function on the dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["startphrase", "ending1", "ending2", "labels", "valid"])

print(f"Keys of tokenized dataset: {list(tokenized_dataset['validation'].features)}")

# save datasets to disk for later easy loading
tokenized_dataset["validation"].save_to_disk("data/eval")


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Validation dataset size: 1094


Map:   0%|          | 0/9674 [00:00<?, ? examples/s]

Map:   0%|          | 0/1094 [00:00<?, ? examples/s]

Map:   0%|          | 0/1146 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['labels', 'input_ids', 'attention_mask']


Saving the dataset (0/1 shards):   0%|          | 0/1094 [00:00<?, ? examples/s]

dataset['validation'].

In [6]:
# Import necessary libraries
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Print out a few examples from the raw dataset
print("Raw Data Examples: ", dataset['validation'][:3])

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Apply preprocessing function to a few examples

dummy = dataset['validation'].select(range(3))

preprocessed_examples = dummy.map(preprocess_function, batched=True, remove_columns=["startphrase", "ending1", "ending2", "labels", "valid"])


# Print out preprocessed examples
print("Preprocessed Data Examples: ", preprocessed_examples)

# Prepare a batch of data
input_ids = torch.tensor([example['input_ids'] for example in preprocessed_examples])
attention_mask = torch.tensor([example['attention_mask'] for example in preprocessed_examples])
labels = torch.tensor([example['labels'] for example in preprocessed_examples])

# Print out a few labels
print("Labels: ", labels)

# Print out model input
print("Model Input: ", {
    'input_ids': input_ids,
    'attention_mask': attention_mask,
    'labels': labels
})

# Decoding input_ids
decoded_input_ids = [tokenizer.decode(input_id) for input_id in input_ids]
print("Decoded Input IDs: ", decoded_input_ids)

# Replace -100 with tokenizer.pad_token_id
labels_replaced = labels.clone()
labels_replaced[labels == -100] = tokenizer.pad_token_id

# Decoding labels
decoded_labels = [tokenizer.decode(label, skip_special_tokens=True) for label in labels_replaced]
print("Decoded Labels: ", decoded_labels)



Raw Data Examples:  {'startphrase': ['The girl had the flightiness of a sparrow', 'The girl had the flightiness of a rock', 'It was as peaceful as a church.'], 'ending1': ['The girl was very fickle.', 'The girl was very fickle.', 'It was very peaceful.'], 'ending2': ['The girl was very stable.', 'The girl was very stable.', 'It was full of conflict and danger, not peace.'], 'labels': [0, 1, 0], 'valid': [1, 1, 1]}


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Preprocessed Data Examples:  Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 3
})
Labels:  tensor([[   37,  3202,    47,   182,   361, 19376,     5,     1,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100],
        [   37,  3202,    47,   182,  5711,     5,     1,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100],
        [   94,    47,   182,  9257,     5,     1,  -100,  -100,  -100,  -100,
     

In [30]:
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# huggingface hub model id

#model_id = "philschmid/flan-t5-xxl-sharded-fp16"
model_id = "google/flan-t5-large"

# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

In [8]:
subset_test_dataset = dataset['validation'].select(range(500))

In [9]:
out_df, preds, labels = evaluate_model(model, tokenizer, subset_test_dataset, verbose = False, total = 500)

100%|██████████| 500/500 [06:15<00:00,  1.33it/s]


In [10]:
zero_shot_accuracy, conf_matrix_zero_shot =  compute_stats(out_df, preds, labels)

overall accuracy: 
0.608
confusion matrix: 
correct forward 304 wrong forward 196 correct backward 346 wrong_backward 154


In [9]:
!pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00


In [12]:
import evaluate
import numpy as np
from datasets import load_from_disk
from tqdm import tqdm

# Metric
metric = evaluate.load("rouge")

# def evaluate_peft_model(sample,max_target_length=50):
#     # generate summary
#     outputs = model.generate(input_ids=sample["input_ids"].unsqueeze(0).cuda(), do_sample=True, top_p=0.9, max_new_tokens=max_target_length)    
#     prediction = tokenizer.decode(outputs[0].detach().cpu().numpy(), skip_special_tokens=True)
#     # decode eval sample
#     # Replace -100 in the labels as we can't decode them.
#     labels = np.where(sample['labels'] != -100, sample['labels'], tokenizer.pad_token_id)
#     labels = tokenizer.decode(labels, skip_special_tokens=True)

#     # Some simple post-processing
#     return prediction, labels

def evaluate_peft_model(sample, max_target_length=50):
    # generate summary
    outputs = model.generate(input_ids=sample["input_ids"].unsqueeze(0).cuda(), do_sample=True, top_p=0.9, max_new_tokens=max_target_length)    
    prediction = tokenizer.decode(outputs[0].detach().cpu().numpy(), skip_special_tokens=True)
    
    # decode eval sample
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(sample['labels'] != -100, sample['labels'], tokenizer.pad_token_id)
    labels = tokenizer.decode(labels, skip_special_tokens=True)

    # Print inputs, predicted summary, and reference summary
    input_text = tokenizer.decode(sample["input_ids"], skip_special_tokens=True)
    print(f"Input: {input_text}")
    print(f"Predicted Summary: {prediction}")
    print(f"Reference Summary: {labels}")
    print("="*50)  # prints a separator

    # Some simple post-processing
    return prediction, labels

# load test dataset from distk
test_dataset = load_from_disk("data/eval/").with_format("torch")

# run predictions
# this can take ~45 minutes
predictions, references = [] , []
for i,sample in tqdm(enumerate(test_dataset)):
    p,l = evaluate_peft_model(sample)
    predictions.append(p)
    references.append(l)
    if i == 20:
        break

# compute metric 
rogue = metric.compute(predictions=predictions, references=references, use_stemmer=True)

# print results 
print(f"Rogue1: {rogue['rouge1']* 100:2f}%")
print(f"rouge2: {rogue['rouge2']* 100:2f}%")
print(f"rougeL: {rogue['rougeL']* 100:2f}%")
print(f"rougeLsum: {rogue['rougeLsum']* 100:2f}%")


# Rogue1: 50.386161%
# rouge2: 24.842412%
# rougeL: 41.370130%
# rougeLsum: 41.394230%

1it [00:02,  2.40s/it]

Input: The girl had the flightiness of a sparrow
Predicted Summary: He would make the bird flee in a moment.
Reference Summary: The girl was very fickle.


2it [00:02,  1.33s/it]

Input: The girl had the flightiness of a rock
Predicted Summary: She looked very young.
Reference Summary: The girl was very stable.


3it [00:03,  1.09it/s]

...

Input: Peace is a human flying
Predicted Summary: Peace is a human flying
Reference Summary: Peace is impossible


20it [00:17,  1.17it/s]

Input: Peace is a human walking
Predicted Summary: , he is a human walking
Reference Summary: Peace is possible


20it [00:17,  1.12it/s]

Input: Plants are a lullaby
Predicted Summary: plantation
Reference Summary: Plants are calming
Rogue1: 16.559615%
rouge2: 5.292784%
rougeL: 16.643501%
rougeLsum: 16.636180%


## 3. Fine-Tune T5 with LoRA and bnb int-8

In addition to the LoRA technique, we will use [bitsanbytes LLM.int8()](https://huggingface.co/blog/hf-bitsandbytes-integration) to quantize out frozen LLM to int8. This allows us to reduce the needed memory for FLAN-T5 XXL ~4x.  

The first step of our training is to load the model. We are going to use [philschmid/flan-t5-xxl-sharded-fp16](https://huggingface.co/philschmid/flan-t5-xxl-sharded-fp16), which is a sharded version of [google/flan-t5-xxl](https://huggingface.co/google/flan-t5-xxl). The sharding will help us to not run off of memory when loading the model.

Now, we can prepare our model for the LoRA int-8 training using `peft`.

In [ ]:
def print_modules(model, prefix=''):
    for name, module in model.named_children():
        full_name = f"{prefix}.{name}" if prefix else name
        print(full_name)
        print_modules(module, full_name)

print_modules(model)

In [31]:
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType

# Define LoRA Config 
lora_config = LoraConfig(
 r=8, 
 lora_alpha=32,
 target_modules=["q", "v"],
 lora_dropout=0.05,
 bias="none",
 task_type=TaskType.SEQ_2_SEQ_LM
)
# prepare int-8 model for training
model = prepare_model_for_int8_training(model)

# add LoRA adaptor
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 2,359,296 || all params: 785,509,376 || trainable%: 0.30035236651331837


/opt/conda/lib/python3.10/site-packages/peft/utils/other.py:107: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


As you can see, here we are only training 0.16% of the parameters of the model! This huge memory gain will enable us to fine-tune the model without memory issues.

Next is to create a `DataCollator` that will take care of padding our inputs and labels. We will use the `DataCollatorForSeq2Seq` from the 🤗 Transformers library.

In [32]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

The last step is to define the hyperparameters (`TrainingArguments`) we want to use for our training.

In [16]:
# Shuffle the training dataset
shuffled_train_dataset = tokenized_dataset["train"].shuffle(seed=42)  

In [33]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, EarlyStoppingCallback
output_dir="lora-flan-t5-large"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=150,
    learning_rate= 0.001,
    max_steps=5000, # ~ 6 epochs
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=20,
    evaluation_strategy="steps",
    eval_steps=20,
    save_strategy="steps",  # Adjust this line to match evaluation_strategy
    save_total_limit=3,  # Optional: the number of total saved models
    report_to="tensorboard",
    load_best_model_at_end=True,
    #lr_scheduler_type="reduce_lr_on_plateau",#cosine
)

# Create Trainer instance with early stopping callback
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset= shuffled_train_dataset,
    eval_dataset=tokenized_dataset["validation"].select(range(100)),
    callbacks=[EarlyStoppingCallback(early_stopping_patience= 3, early_stopping_threshold=0.05)]  # Add early stopping callback here
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

# train model
trainer.train()


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
20,1.813600,1.557061
40,1.508000,1.515142
60,1.490500,1.476398
80,1.410000,1.455266
100,1.366000,1.436849
120,1.384700,1.414466
140,1.317800,1.420349
160,1.301600,1.428977
180,1.277000,1.418620
200,1.280500,1.424251


KeyboardInterrupt: 

Let's now train our model and run the cells below. Note that for T5, some layers are kept in `float32` for stability purposes.

In [41]:
%load_ext tensorboard
%tensorboard --logdir /kaggle/working/lora-flan-t5-base/logs --port 6007

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6007 (pid 461), started 0:00:12 ago. (Use '!kill 461' to kill it.)

The training took ~10:36:00 and cost `~13.22$` for 10h of training. For comparison a [full fine-tuning on FLAN-T5-XXL](https://www.philschmid.de/fine-tune-flan-t5-deepspeed#3-results--experiments) with the same duration (10h) requires 8x A100 40GBs and costs ~322$. 

We can save our model to use it for inference and evaluate it. We will save it to disk for now, but you could also upload it to the [Hugging Face Hub](https://huggingface.co/docs/hub/main) using the `model.push_to_hub` method.

In [34]:
# Save our LoRA model & tokenizer results
peft_model_id="results"
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)
# if you want to save the base model to call
# trainer.model.base_model.save_pretrained(peft_model_id)

('results/tokenizer_config.json',
 'results/special_tokens_map.json',
 'results/spiece.model',
 'results/added_tokens.json',
 'results/tokenizer.json')

Our LoRA checkpoint is only 84MB small and includes all of the learnt knowleddge for samsum.

## 4. Evaluate & run Inference with LoRA FLAN-T5

After the training is done we want to evaluate and test it. The most commonly used metric to evaluate summarization task is [rogue_score](https://en.wikipedia.org/wiki/ROUGE_(metric)) short for Recall-Oriented Understudy for Gisting Evaluation). This metric does not behave like the standard accuracy: it will compare a generated summary against a set of reference summaries.

We are going to use `evaluate` library to evaluate the `rogue` score. We can run inference using `PEFT` and `transformers`. For our FLAN-T5 XXL model, we need at least 18GB of GPU memory.

In [35]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load peft config for pre-trained checkpoint etc. 
peft_model_id = "results"
config = PeftConfig.from_pretrained(peft_model_id)

# load base LLM model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path,  load_in_8bit=True,  device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id, device_map={"":0})
model.eval()

print("Peft model loaded")

Peft model loaded


In [20]:
from huggingface_hub import notebook_login

notebook_login()

In [21]:
# Push the model to your namespace with the name "my-finetuned-bert".
model.push_to_hub("T5-Large-figQA-seq2seq-second-run")

adapter_model.bin:   0%|          | 0.00/9.54M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/davidguzmanp/T5-Large-figQA-seq2seq-second-run/commit/0e902a2e7a690f95ccc9ff0af43990fa759dee17', commit_message='Upload model', commit_description='', oid='0e902a2e7a690f95ccc9ff0af43990fa759dee17', pr_url=None, pr_revision=None, pr_num=None)

Let’s load the dataset again with a random sample to try the summarization.

Nice! our model works! Now, lets take a closer look and evaluate it against the `test` set of processed dataset from `samsum`. Therefore we need to use and create some utilities to generate the summaries and group them together. The most commonly used metrics to evaluate summarization task is [rogue_score](https://en.wikipedia.org/wiki/ROUGE_(metric)) short for Recall-Oriented Understudy for Gisting Evaluation). This metric does not behave like the standard accuracy: it will compare a generated summary against a set of reference summaries.

----

In [37]:
out_df, preds, labels = evaluate_model(model, tokenizer, subset_test_dataset, verbose = False, total = 500)
zero_shot_accuracy, conf_matrix_zero_shot =  compute_stats(out_df, preds, labels)

100%|██████████| 500/500 [10:39<00:00,  1.28s/it]

overall accuracy: 
0.588
confusion matrix: 
correct forward 294 wrong forward 206 correct backward 291 wrong_backward 209


In [36]:
import evaluate
import numpy as np
from datasets import load_from_disk
from tqdm import tqdm

# Metric
metric = evaluate.load("rouge")

# def evaluate_peft_model(sample,max_target_length=50):
#     # generate summary
#     outputs = model.generate(input_ids=sample["input_ids"].unsqueeze(0).cuda(), do_sample=True, top_p=0.9, max_new_tokens=max_target_length)    
#     prediction = tokenizer.decode(outputs[0].detach().cpu().numpy(), skip_special_tokens=True)
#     # decode eval sample
#     # Replace -100 in the labels as we can't decode them.
#     labels = np.where(sample['labels'] != -100, sample['labels'], tokenizer.pad_token_id)
#     labels = tokenizer.decode(labels, skip_special_tokens=True)

#     # Some simple post-processing
#     return prediction, labels

def evaluate_peft_model(sample, max_target_length=50):
    # generate summary
    outputs = model.generate(input_ids=sample["input_ids"].unsqueeze(0).cuda(), do_sample=True, top_p=0.9, max_new_tokens=max_target_length)    
    prediction = tokenizer.decode(outputs[0].detach().cpu().numpy(), skip_special_tokens=True)
    
    # decode eval sample
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(sample['labels'] != -100, sample['labels'], tokenizer.pad_token_id)
    labels = tokenizer.decode(labels, skip_special_tokens=True)

    # Print inputs, predicted summary, and reference summary
    input_text = tokenizer.decode(sample["input_ids"], skip_special_tokens=True)
    print(f"Input: {input_text}")
    print(f"Predicted Summary: {prediction}")
    print(f"Reference Summary: {labels}")
    print("="*50)  # prints a separator

    # Some simple post-processing
    return prediction, labels

# load test dataset from distk
test_dataset = load_from_disk("data/eval/").with_format("torch")

# run predictions
# this can take ~45 minutes
predictions, references = [] , []
for i,sample in tqdm(enumerate(test_dataset)):
    p,l = evaluate_peft_model(sample)
    predictions.append(p)
    references.append(l)
    if i == 20:
        break

# compute metric 
rogue = metric.compute(predictions=predictions, references=references, use_stemmer=True)

# print results 
print(f"Rogue1: {rogue['rouge1']* 100:2f}%")
print(f"rouge2: {rogue['rouge2']* 100:2f}%")
print(f"rougeL: {rogue['rougeL']* 100:2f}%")
print(f"rougeLsum: {rogue['rougeLsum']* 100:2f}%")

# Rogue1: 50.386161%
# rouge2: 24.842412%
# rougeL: 41.370130%
# rougeLsum: 41.394230%

1it [00:01,  1.45s/it]

Input: The girl had the flightiness of a sparrow
Predicted Summary: The girl was very flighty.
Reference Summary: The girl was very fickle.


2it [00:02,  1.33s/it]

Input: The girl had the flightiness of a rock
Predicted Summary: The girl was not flighty
Reference Summary: The girl was very stable.


3it [00:03,  1.24s/it]

...

Input: Peace is a human flying
Predicted Summary: Peace is not intangible
Reference Summary: Peace is impossible


20it [00:24,  1.07s/it]

Input: Peace is a human walking
Predicted Summary: Peace is always present
Reference Summary: Peace is possible


20it [00:25,  1.25s/it]

Input: Plants are a lullaby
Predicted Summary: Plants bring happiness
Reference Summary: Plants are calming
Rogue1: 59.131850%
rouge2: 41.632653%
rougeL: 58.396014%
rougeLsum: 58.658567%


-----

In [38]:
import evaluate
import numpy as np
from datasets import load_from_disk
from tqdm import tqdm

# Metric
metric = evaluate.load("rouge")

# def evaluate_peft_model(sample,max_target_length=50):
#     # generate summary
#     outputs = model.generate(input_ids=sample["input_ids"].unsqueeze(0).cuda(), do_sample=True, top_p=0.9, max_new_tokens=max_target_length)    
#     prediction = tokenizer.decode(outputs[0].detach().cpu().numpy(), skip_special_tokens=True)
#     # decode eval sample
#     # Replace -100 in the labels as we can't decode them.
#     labels = np.where(sample['labels'] != -100, sample['labels'], tokenizer.pad_token_id)
#     labels = tokenizer.decode(labels, skip_special_tokens=True)

#     # Some simple post-processing
#     return prediction, labels

def evaluate_peft_model(sample, max_target_length=50):
    # generate summary
    outputs = model.generate(input_ids=sample["input_ids"].unsqueeze(0).cuda(), do_sample=True, top_p=0.9, max_new_tokens=max_target_length)    
    prediction = tokenizer.decode(outputs[0].detach().cpu().numpy(), skip_special_tokens=True)
    
    # decode eval sample
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(sample['labels'] != -100, sample['labels'], tokenizer.pad_token_id)
    labels = tokenizer.decode(labels, skip_special_tokens=True)

    # Print inputs, predicted summary, and reference summary
    input_text = tokenizer.decode(sample["input_ids"], skip_special_tokens=True)
    print(f"Input: {input_text}")
    print(f"Predicted Summary: {prediction}")
    print(f"Reference Summary: {labels}")
    print("="*50)  # prints a separator

    # Some simple post-processing
    return prediction, labels

# load test dataset from distk
test_dataset = load_from_disk("data/eval/").with_format("torch")

# run predictions
# this can take ~45 minutes
predictions, references = [] , []
for i,sample in tqdm(enumerate(test_dataset)):
    p,l = evaluate_peft_model(sample)
    predictions.append(p)
    references.append(l)
    if i == 50:
        break

# compute metric 
rogue = metric.compute(predictions=predictions, references=references, use_stemmer=True)

# print results 
print(f"Rogue1: {rogue['rouge1']* 100:2f}%")
print(f"rouge2: {rogue['rouge2']* 100:2f}%")
print(f"rougeL: {rogue['rougeL']* 100:2f}%")
print(f"rougeLsum: {rogue['rougeLsum']* 100:2f}%")

# Rogue1: 50.386161%
# rouge2: 24.842412%
# rougeL: 41.370130%
# rougeLsum: 41.394230%

1it [00:01,  1.45s/it]

Input: The girl had the flightiness of a sparrow
Predicted Summary: The girl was very flighty.
Reference Summary: The girl was very fickle.


2it [00:03,  1.63s/it]

Input: The girl had the flightiness of a rock
Predicted Summary: The girl was not flighty at all.
Reference Summary: The girl was very stable.


3it [00:04,  1.39s/it]

...

Input: HIs opinions were as firm as concrete
Predicted Summary: His opinions were well known
Reference Summary: He was very certain of his opinion


50it [01:05,  1.26s/it]

Input: HIs opinions were as firm as a cotton ball
Predicted Summary: His opinions were not firm
Reference Summary: He was very uncertain of his opinion


50it [01:07,  1.34s/it]

Input: The pilot landed the plane like he was handling a new born baby.
Predicted Summary: The pilot landed gently.
Reference Summary: The landing was smooth and gentle
Rogue1: 44.545694%
rouge2: 27.804075%
rougeL: 44.296814%
rougeLsum: 44.620021%


Our PEFT fine-tuned FLAN-T5-XXL achieved a rogue1 score of `50.38%` on the test dataset. For comparison a [full fine-tuning of flan-t5-base achieved a rouge1 score of 47.23](https://www.philschmid.de/fine-tune-flan-t5). That is a `3%` improvements. 

It is incredible to see that our LoRA checkpoint is only 84MB small and model achieves better performance than a smaller fully fine-tuned model.